In [1]:
import os 
os.chdir("../..")
from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)
os.chdir("Quai-Macro-Model/Spec")

from IPython.display import display, display_html
from copy import deepcopy
from src import math_spec_json
import numpy as np
from pprint import pprint
from simulation import state_base, params_base, vesting_schedule_translate, build_logistic_classifier
import pandas as pd

ms = load_from_json(deepcopy(math_spec_json))
msi = ms.build_implementation(params_base)

Placeholder has no control action options


## Sanity Check Scenario

Premise: Ensure the simulation framework is operating as intended, without complications from the controller.

Condition(s): Fix the values of:
- $\mathbf u$, the controller update parameters ($k_{qi}, k_{quai}$); and
- $\pmb{\beta}$, the _miner decision population parameter vector_, i.e. the parameter vector for the miner's probabilistic decision on which token to receive as block reward.

Method: MC runs (across realizations of other stochastic variables), generating time series for e.g. miner decisions and proposed block rewards. 

In [12]:
state, params = msi.prepare_state_and_params(state_base, params_base, state_preperation_functions=[vesting_schedule_translate,
                                                                                                   build_logistic_classifier], parameter_preperation_functions=[])


for _ in range(10000):
    # Set fixed values
    state["K Qi"] = 1
    state["K Quai"] = 500
    state["Population Mining Beta Vector"] = np.array([0, .001])


    msi.wiring["Mine Block Wiring"](state, params, [])
    msi.mechanisms["Log Simulation Data Mechanism"](state, params, [])

df = pd.DataFrame(state["Simulation History Log"])

In [13]:
df

,Estimate Beta0,Estimate Beta1,Population Beta0,Population Beta1
0,14.910537,-296.126490,0.0,0.001
1,4.935834,2852.238486,0.0,0.001
2,-14.705882,-1049.309965,0.0,0.001
3,-14.605648,-3609.575863,0.0,0.001
4,14.506770,-1333.606497,0.0,0.001
...,...,...,...,...
9995,-94.252815,-43.895290,0.0,0.001
9996,-93.961742,-22.370196,0.0,0.001
9997,-93.952476,24.864504,0.0,0.001
9998,-93.943212,22.147882,0.0,0.001


## Sanity Check B

Above sanity check but with changing difficulties

In [14]:
state, params = msi.prepare_state_and_params(state_base, params_base, state_preperation_functions=[vesting_schedule_translate,
                                                                                                   build_logistic_classifier], parameter_preperation_functions=[])

for _ in range(10000):
    # Set fixed values
    state["K Qi"] = 1
    state["K Quai"] = 500
    state["Population Mining Beta Vector"] = np.array([0, 0.001])
    state["Block Difficulty"] = max(np.random.normal(6000, 1000), 100)


    msi.wiring["Mine Block Wiring"](state, params, [])
    msi.mechanisms["Log Simulation Data Mechanism"](state, params, [])
df = pd.DataFrame(state["Simulation History Log"])

In [15]:


df

,Estimate Beta0,Estimate Beta1,Population Beta0,Population Beta1
0,24.850895,1177.571120,0.0,0.001
1,14.807502,907.245386,0.0,0.001
2,24.509804,2332.524928,0.0,0.001
3,4.868549,2163.148504,0.0,0.001
4,-4.835590,-2353.591170,0.0,0.001
...,...,...,...,...
9995,-126.892942,65.700999,0.0,0.001
9996,-127.162205,-13.740738,0.0,0.001
9997,-127.008790,3.120818,0.0,0.001
9998,-127.137128,-58.434643,0.0,0.001
